A few things you should keep in mind when working on assignments:

1. Make sure you fill in any place that says `YOUR CODE HERE`. Do **not** write your answer in anywhere else other than where it says `YOUR CODE HERE`. Anything you write anywhere else will be removed or overwritten by the autograder.

2. Before you submit your assignment, make sure everything runs as expected. Go to menubar, select _Kernel_, and restart the kernel and run all cells (_Restart & Run all_).

3. Do not change the title (i.e. file name) of this notebook.

4. Make sure that you save your work (in the menubar, select _File_ → _Save and CheckPoint_)

5. You are allowed to submit an assignment multiple times, but only the most recent submission will be graded.

# Problem 1. Cassandra

In this problem, we use the Cassandra Python database driver to execute CQL (Cassandra Query Language) queries.

In [ ]:
import os
import cassandra
from cassandra.cluster import Cluster
from cassandra.policies import WhiteListRoundRobinPolicy
from cassandra.query import dict_factory
from cassandra.cqlengine import connection, management

from nose.tools import assert_equal, assert_true, assert_is_instance, assert_almost_equal

Suppose we are given the following table:

<div class="row">
    <div class="col-md-2">
      <div align="center">
        <b>Midterm</b>
      </div>
    <table>
  <tr>
    <th>Id</th>
    <th>Name</th> 
    <th>Score</th>
  </tr>
  <tr>
    <td>1</td>
    <td>Alice</td> 
    <td>97.3</td>
  </tr>
  <tr>
    <td>2</td>
    <td>Bob</td> 
    <td>87.7</td>
  </tr>
  <tr>
    <td>3</td>
    <td>Chris</td> 
    <td>91.5</td>
  </tr>
</table>
    </div>
</div>

We can represent this table as a list of dictionaries, as shown in the following code cell.

In [ ]:
midterm = [
    {"Id": 1, "Name": "Alice", "Score": 97.3},
    {"Id": 2, "Name": "Bob", "Score": 87.7},
    {"Id": 3, "Name": "Chris", "Score": 91.5}
]

We connect to the course Cassandra server as we did in the [Introduction to Cassandra notebook](https://github.com/UI-DataScience/accy571-fa16/blob/master/Week14/notebooks/intro2cassandra.ipynb).

In [ ]:
cassandra_ips = ['192.168.100.24']

# Establish a connection to Cassandra

# The Policy is necessary to allow Cassandra to run on Azure.
from cassandra.policies import WhiteListRoundRobinPolicy
pcy = WhiteListRoundRobinPolicy(cassandra_ips)

# Create Connection
cluster = Cluster(contact_points=cassandra_ips, load_balancing_policy=pcy)
session = cluster.connect()

print('Cluster Name: {0}'.format(cluster.metadata.cluster_name))
for host in cluster.metadata.all_hosts():
    print('{0}: Host: {1} in {2}'\
          .format(host.datacenter, host.address, host.rack))

To provide distinct environments for each student, each student will create their own keyspace in the shared Cassandra cluster by using their NetIDs.

In [ ]:
# Filename containing user's netid
fname = '/home/data_scientist/users.txt'
with open(fname, 'r') as fin:
    netid = fin.readline().rstrip()

# We will delete our working directory if it exists before recreating.
ks_name = '{0}'.format(netid)

session.row_factory = dict_factory

connection.set_session(session)

# Explicitly set session hosts, this removes annoying warnings.
connection.session.hosts = cassandra_ips

# Drop Keyspace if it exists
if ks_name in cluster.metadata.keyspaces:
    management.drop_keyspace(ks_name)

# Create Keyspace
management.create_keyspace_simple(ks_name, 1)

# Set keyspace for this session
# Note: If keyspace exists in Cassandra instance, this is only line we need.
session.set_keyspace(ks_name)

# Display all non-system keyspaces.
# Do not change to a different keyspace!

keys = [val for val in sorted(cluster.metadata.keyspaces.keys()) if 'system' not in val]
for ks in keys:
    print(ks)

print('\nCQL Query to recreate this keyspace:')
print(40*'-')
print(cluster.metadata.keyspaces[ks_name].export_as_string())

We first drop the table if it exists to ensure a clean slate before we create our new schema and insert data. Note that the table name we will be using in this problem is `midterm`.

In [ ]:
def drop_table(session, table):
    """
    Drops a table if exists.
    
    Parameters
    ----------
    session: A cassandra.cluster.Session instance.
    table: A string. Table name.
    
    Returns
    -------
    A cassandra.cluster.ResultSet instance.
    """
    
    result = session.execute("DROP TABLE IF EXISTS {};".format(table))
    return result

result = drop_table(session, "midterm")

assert_is_instance(result, cassandra.cluster.ResultSet)
assert_equal(result.column_names, None)
assert_equal(len(result.current_rows), 0)

## Create a table

- Complete the following function by writing a CQL schema that creates a table named `midterm` that has the following 3 columns:
  - `id` (`int`, `PRIMARY KEY`)
  - `name` (`text`)
  - `score` (`float`)
  
Hints:

- The _Schema Creation_ section of the [Introduction to Cassandra notebook](https://github.com/UI-DataScience/accy571-fa16/blob/master/Week14/notebooks/intro2cassandra.ipynb) has some examples. See also the [official documentation](http://docs.datastax.com/en/cql/3.3/cql/cql_using/useCreateTableTOC.html).

- One item of importance is the creation of the [primary key](http://docs.datastax.com/en/cql/3.3/cql/cql_using/useSimplePrimaryKeyConcept.html#useSimplePrimaryKeyConcept). Use `id` as the primary key.

In [ ]:
def create_table(session):
    # YOUR CODE HERE

    result = session.execute(create_schema)
    
    return result

In [ ]:
create_result = create_table(session)
print(create_result.response_future)

In [ ]:
result = session.execute("SELECT * FROM Midterm;")
assert_is_instance(result, cassandra.cluster.ResultSet)
assert_equal(set(result.column_names), {"id", "name", "score"})
assert_equal(len(result.current_rows), 0)

## Insert data

- Complete the following function by writing a **prepared statement** that inserts data into the `midterm` table.

Hints:

- The _Adding Data_ section of the [Introduction to Cassandra notebook](https://github.com/UI-DataScience/accy571-fa16/blob/master/Week14/notebooks/intro2cassandra.ipynb) has some examples. See also the official documentation [here](https://docs.datastax.com/en/developer/java-driver/3.1/manual/statements/prepared/) and [here](https://datastax.github.io/python-driver/api/cassandra/cluster.html#cassandra.cluster.Session.prepare).


In [ ]:
def insert_data(session, data):
    
    # YOUR CODE HERE
    prepared = session.prepare(insert_many)

    for idx, student in enumerate(data):
        result = session.execute(prepared, (idx + 1, student['Name'], student['Score']))
    
    return result

In [ ]:
insert_result = insert_data(session, midterm)
print(insert_result.response_future)

In [ ]:
result = session.execute("SELECT * FROM midterm;")
assert_is_instance(result, cassandra.cluster.ResultSet)
assert_equal(len(result.current_rows), len(midterm))
assert_equal(
    {row["name"] for row in result.current_rows},
    {student["Name"] for student in midterm}
    )
assert_equal(
    {row["id"] for row in result.current_rows},
    set(range(1, len(midterm) + 1))
    )

## Retrieve data

- The following function accepts an argument named `id_`. It uses this argument in a CQL prepared statement, queries the matching `id` column in the Cassandra table, and then selects the `name` and `score` columns. Complete this function by writing a prepared statement that selects the `name` and `score` of a student by matching the `id` column.

Hints:

- When you run
  ```python
  >>> retrieve_result = retrieve(session, 1)
  >>> print(retrieve_result.current_rows)
  ```
  you should get
  ```
  [{'name': 'Alice', 'score': 97.30000305175781}]
  ```
  Similary,
  ```python
  >>> retrieve_result = retrieve(session, 2)
  >>> print(retrieve_result.current_rows)
  ```
  ```
  [{'name': 'Bob', 'score': 87.69999694824219}]
  ```
  ```python
  >>> retrieve_result = retrieve(session, 3)
  >>> print(retrieve_result.current_rows)
  ```
  ```
  [{'name': 'Chris', 'score': 91.5}]
  ```
  
- The _Retrieving Data_ section of the [Introduction to Cassandra notebook](https://github.com/UI-DataScience/accy571-fa16/blob/master/Week14/notebooks/intro2cassandra.ipynb) has some examples. See also the official documentation [here](https://datastax.github.io/python-driver/api/cassandra/cluster.html#cassandra.cluster.Session.prepare).

In [ ]:
def retrieve(session, id_):
    # YOUR CODE HERE
    result = session.execute(query, {"id_": id_})
    
    return result

In [ ]:
retrieve_result = retrieve(session, 1)
print(retrieve_result.current_rows)

In [ ]:
retrieve_result = retrieve(session, 2)
print(retrieve_result.current_rows)

In [ ]:
retrieve_result = retrieve(session, 3)
print(retrieve_result.current_rows)

In [ ]:
assert_is_instance(result, cassandra.cluster.ResultSet)

def get_answer(table):
    return [{"name": s["Name"], "score": s["Score"]} for s in table if s["Id"] == 1]

def get_names(items):
    return [i["name"] for i in items]

def get_scores(items):
    return [i["score"] for i in items]

def test_names(session, id_, table):
    t = retrieve(session, id_).current_rows
    a = get_answer(table)
    t_names = get_names(t)
    a_names = get_names(a)
    assert_equal(len(t_names), len(a_names))
    assert_equal(set(a_names), set(a_names))

def test_scores(session, id_, table):
    t = retrieve(session, id_).current_rows
    a = get_answer(table)
    t_scores = get_scores(t)
    a_scores = get_scores(a)
    assert_equal(len(t_scores), len(a_scores))
    assert_equal(set(a_scores), set(a_scores))
    
for i in range(1, 4):
    test_names(session, i, midterm)
    test_scores(session, i, midterm)